<a href="https://colab.research.google.com/github/qiongwu-git/my_project/blob/master/Google_ML_Intro_to_fairness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is made to practice Google ML chapter "intro_to_fairness". The original script was done with Tensorflow, but in this script we will take PyTorch and make some tweaks here and there. So buckle up!

In [5]:
# Import libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve
import torch
import torch.nn as nn

print("Modules are imported.")

Modules are imported.


In [10]:
# Load the dataset
COLUMNS = ["age", "workclass", "fnlwgt", "education", "education_num",
           "marital_status", "occupation", "relationship", "race", "gender",
           "capital_gain", "capital_loss", "hours_per_week", "native_country",
           "income_bracket"]

train_df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    names=COLUMNS,
    sep=r'\s*,\s*',
    engine='python',
    na_values="?")

test_df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
    names=COLUMNS,
    sep=r'\s*,\s*',
    skiprows=[0],
    engine='python',
    na_values="?")

# Drop rows with missing values
train_df = train_df.dropna(how="any", axis=0)
test_df = test_df.dropna(how="any", axis=0)

print('UCI Adult Census Income dataset loaded.')

UCI Adult Census Income dataset loaded.


In [7]:
# Tweak no.1: the original script uses Facets Overview here to describe the dataset. But another approach is to use pandas profiling, which will also provide some useful features.
import pandas_profiling
pandas_profiling.ProfileReport(train_df)


/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,16
Number of observations,30162
Total Missing (%),0.0%
Total size in memory,3.7 MiB
Average record size in memory,128.0 B
Numeric,7
Categorical,9
Boolean,0
Date,0
Text (Unique),0
Rejected,0




1.   Task 1: are there missing values for a large number of oberservations?

    The "**total missing**" info writes "**0.0%**", so there are no missing values and it is not a concern here.
2.   Task 2: are there features that are missing that might affect other features?
3.   Task 3: are there any unexpected feature values?
4.   Task 4: what signs of data skew do you see?



*   Gender: male (67.6%) vs. female (32.4%) This suggests strong datq skew.
*   Capital gain: 91.6% are 0's.
*   Capital loss: 95.3% are 0's.
    It is certainly possible for such cases to exist, since capital gains/losses are only registered for individuals for make investments. But it doesn't mean we should just take the values as they are. We may want to take a closer look later to validate it.
*   Hours per week: 7 people reported that they work only 1 hour per week. Is it plausible?

5.   Dive deeper: let's check out the correlations!
   We may want to exclude capital loss and capital gains since in this case, they are not contributing too much to our analysis. The original script decided to omit these two altogether, but there is another idea, we can turn them into binary variables to suggest whether the individuals are making investments. For consistency's sake, here I will also omit these two variables.






In [23]:
# Tweak no.2: Data processing
train_df.columns
train_df.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education_num      int64
marital_status    object
occupation        object
relationship      object
race              object
gender            object
capital_gain       int64
capital_loss       int64
hours_per_week     int64
native_country    object
income_bracket    object
dtype: object

In [0]:
train_df['age_buckets'] = pd.cut(train_df['age'], 9) # cut the age and make them into bins, so that we can convert them into categotical later.

In [34]:
categorical_columns = ['age_buckets', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'gender', 'native_country', 'income_bracket']
numerical_columns = ['fnlwgt', 'education_num', 'hours_per_week']
outputs = ['income_bracket']
for category in categorical_columns:
  train_df[category] = train_df[category].astype('category')

train_df.dtypes

age                  int64
workclass         category
fnlwgt               int64
education         category
education_num        int64
marital_status    category
occupation        category
relationship      category
race              category
gender            category
capital_gain         int64
capital_loss         int64
hours_per_week       int64
native_country    category
income_bracket    category
age_buckets       category
dtype: object

In [35]:
# Next up we still have to transform the categorical columns into numeric types.
age_buckets = train_df['age_buckets'].cat.codes.values
workclass = train_df['workclass'].cat.codes.values
education = train_df['education'].cat.codes.values
marital_status = train_df['marital_status'].cat.codes.values
occupation = train_df['occupation'].cat.codes.values
relationship = train_df['relationship'].cat.codes.values
race = train_df['race'].cat.codes.values
gender = train_df['gender'].cat.codes.values
native_country = train_df['native_country'].cat.codes.values
income_bracket = train_df['income_bracket'].cat.codes.values

categorical_data = np.stack([age_buckets, workclass, education, marital_status, occupation, relationship, race, gender, native_country], 1)
categorical_data[:10]

array([[ 2,  5,  9,  4,  0,  1,  4,  1, 38],
       [ 4,  4,  9,  2,  3,  0,  4,  1, 38],
       [ 2,  2, 11,  0,  5,  1,  4,  1, 38],
       [ 4,  2,  1,  2,  5,  0,  2,  1, 38],
       [ 1,  2,  9,  2,  9,  5,  2,  0,  4],
       [ 2,  2, 12,  2,  3,  5,  4,  0, 38],
       [ 3,  2,  6,  3,  7,  1,  2,  0, 22],
       [ 4,  4, 11,  2,  3,  0,  4,  1, 38],
       [ 1,  2, 12,  4,  9,  1,  4,  0, 38],
       [ 3,  2,  9,  2,  3,  0,  4,  1, 38]], dtype=int8)

In [31]:
categorical_data = torch.tensor(categorical_data, dtype = torch.int64)
categorical_data[:10]

tensor([[ 2,  5,  9,  4,  0,  1,  4,  1, 38],
        [ 4,  4,  9,  2,  3,  0,  4,  1, 38],
        [ 2,  2, 11,  0,  5,  1,  4,  1, 38],
        [ 4,  2,  1,  2,  5,  0,  2,  1, 38],
        [ 1,  2,  9,  2,  9,  5,  2,  0,  4],
        [ 2,  2, 12,  2,  3,  5,  4,  0, 38],
        [ 3,  2,  6,  3,  7,  1,  2,  0, 22],
        [ 4,  4, 11,  2,  3,  0,  4,  1, 38],
        [ 1,  2, 12,  4,  9,  1,  4,  0, 38],
        [ 3,  2,  9,  2,  3,  0,  4,  1, 38]])

In [33]:
numerical_data = np.stack([train_df[col].values for col in numerical_columns], 1)
numerical_data = torch.tensor(numerical_data, dtype = torch.float)
numerical_data[:5]

tensor([[7.7516e+04, 1.3000e+01, 4.0000e+01],
        [8.3311e+04, 1.3000e+01, 1.3000e+01],
        [2.1565e+05, 9.0000e+00, 4.0000e+01],
        [2.3472e+05, 7.0000e+00, 4.0000e+01],
        [3.3841e+05, 1.3000e+01, 4.0000e+01]])

In [37]:
outputs = torch.tensor(income_bracket, dtype = torch.int64)
outputs[:5]

tensor([0, 0, 0, 0, 0])

In [38]:
print(categorical_data.shape)
print(numerical_data.shape)
print(outputs.shape)

(30162, 9)
torch.Size([30162, 3])
torch.Size([30162])


# Embedding
Instead of using the categorical data to train our model directly, we are going to create a categorical vector. A vector is capable of capturing more information and can find relationships between different categorical values in a more appropraite way. This process is called embedding.

We still have yet to define the embedding size for all the categorical columns. A good rule of thumb is to devide the number of unique values in the column by 2 (but not exceeding 50).

In [54]:
categorical_column_sizes = [len(train_df[column].cat.categories) for column in categorical_columns]
categorical_embedding_sizes = [(col_size, min(50, (col_size+1)//2)) for col_size in categorical_column_sizes]
print(categorical_embedding_sizes)

[(9, 5), (7, 4), (16, 8), (7, 4), (14, 7), (6, 3), (5, 3), (2, 1), (41, 21), (2, 1)]


In [43]:
# Prepare the test dataset.
test_df.columns
test_df['age_buckets'] = pd.cut(test_df['age'], 9)
for category in categorical_columns:
  test_df[category] = test_df[category].astype('category')

age_buckets_test = test_df['age_buckets'].cat.codes.values
workclass_test = test_df['workclass'].cat.codes.values
education_test = test_df['education'].cat.codes.values
marital_status_test = test_df['marital_status'].cat.codes.values
occupation_test = test_df['occupation'].cat.codes.values
relationship_test = test_df['relationship'].cat.codes.values
race_test = test_df['race'].cat.codes.values
gender_test = test_df['gender'].cat.codes.values
native_country_test = test_df['native_country'].cat.codes.values
income_bracket_test = test_df['income_bracket'].cat.codes.values

categorical_test_data = np.stack([age_buckets_test, workclass_test, education_test, marital_status_test, occupation_test, relationship_test, race_test, gender_test, native_country_test], 1)
categorical_test_data = torch.tensor(categorical_test_data, dtype = torch.int64)

numerical_test_data = np.stack([test_df[col].values for col in numerical_columns], 1)
numerical_test_data = torch.tensor(numerical_test_data, dtype = torch.float)

test_outputs = torch.tensor(income_bracket_test, dtype = torch.int64)

print("Test dataset is all set!")

Test dataset is all set!


In [44]:
categorical_train_data = categorical_data
numerical_train_data = numerical_data
train_outputs = outputs

print("Train dataset is all set!")

Train dataset is all set!


In [0]:
# Creating a model for prediction
class Model(nn.Module): # here we create a class named Model, which will be used to train the model.

    def __init__(self, embedding_size, num_numerical_cols, output_size, layers, p=0.4):   
        super().__init__()
        self.all_embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in embedding_size])
        self.embedding_dropout = nn.Dropout(p)
        self.batch_norm_num = nn.BatchNorm1d(num_numerical_cols)

        all_layers = []
        num_categorical_cols = sum((nf for ni, nf in embedding_size))
        input_size = num_categorical_cols + num_numerical_cols

        for i in layers:
            all_layers.append(nn.Linear(input_size, i)) # Linear: used to calculate the dot product between the inputs and weight matrixes
            all_layers.append(nn.ReLU(inplace=True)) # ReLu: which is applied as an activation function
            all_layers.append(nn.BatchNorm1d(i)) # BatchNorm1d: used to apply barch normalization to the numerical columns
            all_layers.append(nn.Dropout(p)) # Dropout: used to avoid overfitting
            input_size = i

        all_layers.append(nn.Linear(layers[-1], output_size))

        self.layers = nn.Sequential(*all_layers)

    def forward(self, x_categorical, x_numerical):
        embeddings = []
        for i,e in enumerate(self.all_embeddings):
            embeddings.append(e(x_categorical[:,i]))
        x = torch.cat(embeddings, 1)
        x = self.embedding_dropout(x)

        x_numerical = self.batch_norm_num(x_numerical)
        x = torch.cat([x, x_numerical], 1)
        x = self.layers(x)
        return x

In [52]:
# Train the model
model = Model(categorical_embedding_sizes, numerical_data.shape[1], 2, [200, 100, 50], p = 0.4)
# We pass the embedding size of the categorical columns, the number of numerical columns, and output size (2 in our case) and the neurons in the hidden layers. You can also choose any other size you want.
print(model)

Model(
  (all_embeddings): ModuleList(
    (0): Embedding(9, 5)
    (1): Embedding(7, 4)
    (2): Embedding(16, 8)
    (3): Embedding(7, 4)
    (4): Embedding(14, 7)
    (5): Embedding(6, 3)
    (6): Embedding(5, 3)
    (7): Embedding(2, 1)
    (8): Embedding(41, 21)
    (9): Embedding(2, 1)
  )
  (embedding_dropout): Dropout(p=0.4, inplace=False)
  (batch_norm_num): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): Linear(in_features=60, out_features=200, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=200, out_features=100, bias=True)
    (5): ReLU(inplace=True)
    (6): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.4, inplace=False)
    (8): Linear(in_features=100, out_features=50, bias=True)
    (9): ReLU(inplace=True)
 

In [0]:
# Loss function and the optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [61]:
epochs = 300
aggregated_losses = []

for i in range(epochs):
    i += 1
    y_pred = model(categorical_train_data, numerical_train_data)
    single_loss = loss_function(y_pred, train_outputs)
    aggregated_losses.append(single_loss)

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

    optimizer.zero_grad()
    single_loss.backward()
    optimizer.step()

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

TypeError: ignored